## JSON

es una estructura de información similar a xml. Ejemplo:
```
data = {
    "key1": "value1",
    "key2": 2,
    "key3": true,
    "key4": null,
    "key5": [
        "value51",
        "value52"
    ],
    "key6": {
        "key61": "value61",
        "key62": "value62"
    }
}
    
```

In [ ]:
json_raw = """{
    "key1": "value1",
    "key2": 2,
    "key3": true,
    "key4": null,
    "key5": [
        "value51",
        "value52"
    ],
    "key6": {
        "key61": "value61",
        "key62": "value62"
    }
}
"""

import json
data = json.loads(json_raw)  # json -> dict

print('1 :', data["key1"] )
print('2 :', data["key2"] )
print('3 :', data["key3"] )
print('4 :', data["key4"] )
print('5 :' , data["key5"] )
print('51:', data["key5"][0] )
print('52:', data["key5"][1] )
print('6 :' , data["key6"] )
print('61:', data["key6"]["key61"] )
print('62:', data["key6"]["key62"] )

data_json = json.dumps(data)  # dict -> json
print(data_json)

___
# API REST
___
Para consumir servicios API REST (los que devuelven un json) instalamos la libreía `requests`:

In [ ]:
! pip install requests --trusted-host pypi.python.org

___
y la importamos:

In [ ]:
import requests

___
## GET
consumir un servicio REST dada su url:

In [ ]:
url = "https://aws.random.cat/meow"
response = requests.get(url)

print(response.status_code)
print('1:', response.text)

# file = json.loads(response.text)['file']
file = response.json()['file']
print('2:', file)

from IPython.display import Image  # https://ipython.org/ipython-doc/dev/api/generated/IPython.display.html
Image(file, width=300, height=300)

___
hacer un GET pasando parámetros:
                          

In [ ]:
# url = "https://en.wikipedia.org/w/api.php?format=json&action=query&prop=extracts&exintro=&explaintext=&titles=pizza"  # Bad
# url = "https://en.wikipedia.org//w/api.php"  # Good

url = "https://test.wikipedia.org//w/api.php"  # mejor usar el entono de pruebas de wikipedia
params = {
    'format': 'json',
    'action': 'query',
    'prop': 'extracts',
    'exintro': '',
    'explaintext': '',
    'titles': 'Page550',
}

response = requests.get(url, params=params)

print( list( response.json()['query']['pages'].values() )[0]['extract'] )

from IPython.display import JSON  # https://ipython.org/ipython-doc/dev/api/generated/IPython.display.html
JSON(response.json())

In [ ]:
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?q=election&api-key=yourkey"

response = requests.get(url)

print(response.json())

___
## POST

Se utiliza POST para enviar un documento (generalmente json) junto con la url:

In [ ]:
url = "http://dummy.restapiexample.com/api/v1/create" 
data = '{"name":"test","salary":"123","age":"23"}'  # json

response = requests.post(url, data=data)

# o
url = "http://dummy.restapiexample.com/api/v1/create" 
data = {'name': 'test', 'salary': 123, 'age': 23}  # dict

response = requests.post(url, json=data)

___
Algunos servicios requieren headers o autorización en la llamada:

In [ ]:
url = "http://dummy.restapiexample.com/api/v1/create" 
headers = {'type': 'application/json',
           'token': 'KJbJBUOu6TGuyUYGG'}

from requests.auth import HTTPBasicAuth
auth = HTTPBasicAuth('<user>', '<pass>')

data = {'name': 'test', 
        'salary': 123, 
        'age': 23}

response = requests.post(url, headers=headers, auth=auth, json=data)

___
# API SOAP
___
Para consumir servicios API SOAP (los que devuelven un xml) instalamos la libreía `zeep`:

In [ ]:
! pip install zeep --trusted-host pypi.python.org

___
examinar un endpoint:

In [ ]:
! python -mzeep http://www.dneonline.com/calculator.asmx?wsdl

In [ ]:
from zeep import Client

client = Client('http://www.dneonline.com/calculator.asmx?wsdl')
result = client.service.Add(2, 1)  # 2 + 1
print('1:', result)

result = client.service.Subtract(intA=2, intB=1)  # 2 - 1
print('2:', result)

result = client.service['Multiply'](intA=2, intB=1)  # 2 * 1
print('3:', result)

___
si quiero saber que xml estoy enviando realmente:

In [ ]:
request = client.create_message(client.service, 'Add', intA=2, intB=1)

from zeep.wsdl.utils import etree_to_string

request_xml = etree_to_string(request).decode()
print(request_xml.replace('><', '>\n<'))

___
si se quiere el xml tal cual lo responde el servicio:

In [ ]:
with client.settings(raw_response=True):
    result = client.service.Add(2, 1)
    
print(result.text.replace('><', '>\n<'))